done

In [ ]:
ARCHITECTURE = 'MLP'
EXPERIMENT_TYPE = 'fixed-eps'
ITERATIONS = 1

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf


from helperfiles.helpers import *
import helperfiles.experiment as experiment



#tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 4.5)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)


In [ ]:
experiment.run(
    structure='unstructured', 
    method='random', 
    scope='local', 
    iterations=ITERATIONS,
    architecture = ARCHITECTURE,
    experiment_type = EXPERIMENT_TYPE,
)

In [ ]:
experiment.run(
    
    structure='unstructured', 
    method='magnitude', 
    scope='local', 
    iterations=ITERATIONS,
    architecture = ARCHITECTURE,
    experiment_type = EXPERIMENT_TYPE,
)

In [ ]:
experiment.run(
    structure='unstructured', 
    method='magnitude', 
    scope='global', 
    iterations=ITERATIONS,
    architecture = ARCHITECTURE,
    experiment_type = EXPERIMENT_TYPE,
)

In [ ]:
experiment.run(
    structure='structured', 
    method='random', 
    scope='local', 
    prune_dense_layers=True,
    iterations=ITERATIONS,
    architecture = ARCHITECTURE,
    experiment_type = EXPERIMENT_TYPE,
)

In [ ]:
experiment.run(
    structure='structured', 
    method='magnitude', 
    scope='local', 
    prune_dense_layers=True,
    iterations=ITERATIONS,
    architecture = ARCHITECTURE,
    experiment_type = EXPERIMENT_TYPE,
)

In [ ]:
experiment.run(
    structure='structured', 
    method='magnitude', 
    scope='global', 
    prune_dense_layers=True,
    iterations=ITERATIONS,
    architecture = ARCHITECTURE,
    experiment_type = EXPERIMENT_TYPE,
)

In [ ]:
@tf.function
def load_image(datapoint):
    input_image, label = normalize(datapoint)
    return input_image, label
ratio='100%'
ds, info = tfds.load(name='mnist', with_info=True, split=[f"train[:{ratio}]",f"test[:{ratio}]"])
ds_train=ds[0]
ds_test=ds[1]
        
def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.reshape(y['image'],(28*28,1)), y['label'])
    return y
ds_test = list(ds_test.map(load_image))
ds_train = list(ds_train.map(load_image))

x_train = tf.convert_to_tensor([sample[0] for sample in ds_train])
y_train = tf.convert_to_tensor([sample[1] for sample in ds_train])
x_test = tf.convert_to_tensor([sample[0] for sample in ds_test])
y_test = tf.convert_to_tensor([sample[1] for sample in ds_test])

In [ ]:
model = initialize_base_model('CNN', [x_train, y_train], 0, '')

In [ ]:
bb0_attack('CNN', model, x_test, y_train)
